# Conversation Management & Classification using Groq API

## Assignment Overview
This notebook implements two core tasks using Groq APIs with OpenAI SDK compatibility:
1. **Task 1:** Managing Conversation History with Summarization
2. **Task 2:** JSON Schema Classification & Information Extraction

**Author:** Kunal Sharma
**Date:** 25-09-2025
---


## 1. Environment Setup and Dependencies
Installing required libraries and importing necessary modules.


In [1]:
# Install required libraries
!pip install openai -q

# Import necessary libraries
import json
import os
from typing import List, Dict, Any, Optional, Tuple
from datetime import datetime
import re
from openai import OpenAI
from pprint import pprint
import time

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


## 2. Groq API Configuration
Setting up the Groq API client using OpenAI-compatible SDK.


In [2]:
# Groq API Configuration
GROQ_API_KEY = "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"

# Initializing Groq client with OpenAI compatibility
client = OpenAI(
    api_key=GROQ_API_KEY,
    base_url="https://api.groq.com/openai/v1"
)

# Available Groq models
AVAILABLE_MODELS = {
    "llama": "llama-3.1-8b-instant",
    "mixtral": "mixtral-8x7b-32768",
    "gemma": "gemma-7b-it"
}

# Using the updated model
MODEL = AVAILABLE_MODELS["llama"]

# Testing the connection
try:
    # Simple test call
    test_response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": "Hello"}],
        max_tokens=10
    )
    print("✅ Groq API connection successful!")
    print(f"Model being used: {MODEL}")
    print(f"Test response: {test_response.choices[0].message.content}")
except Exception as e:
    print(f"❌ Error connecting to Groq API: {str(e)}")
    print("Please check your API key and try again.")


✅ Groq API connection successful!
Model being used: llama-3.1-8b-instant
Test response: Hello, how are you today?


## 3. Task 1: Managing Conversation History with Summarization

### 3.1 Core Classes and Functions
Implementing the conversation management system with customizable truncation and periodic summarization.


In [3]:
class ConversationManager:
    """
    Manages conversation history with summarization capabilities.
    Supports truncation by turns, length, and periodic summarization.
    """

    def __init__(self, model: str = MODEL, summarize_every_k: int = 3):
        """
        Initialize the conversation manager.

        Args:
            model: The Groq model to use
            summarize_every_k: Perform summarization after every k conversations
        """
        self.model = model
        self.conversation_history = []
        self.summarized_history = ""
        self.conversation_count = 0
        self.summarize_every_k = summarize_every_k
        self.all_summaries = []  # Track all summaries for demonstration

    def add_message(self, role: str, content: str):
        """Add a message to the conversation history."""
        self.conversation_history.append({
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        })

    def get_conversation_for_api(self, max_turns: Optional[int] = None,
                                max_chars: Optional[int] = None) -> List[Dict[str, str]]:
        """
        Get conversation history formatted for API calls with truncation options.

        Args:
            max_turns: Maximum number of conversation turns to include
            max_chars: Maximum character length for the conversation
        """
        # Starting with summarized history if exists
        messages = []
        if self.summarized_history:
            messages.append({
                "role": "system",
                "content": f"Previous conversation summary: {self.summarized_history}"
            })

        # Getting recent messages
        recent_messages = self.conversation_history
        if max_turns:
            recent_messages = recent_messages[-max_turns:]

        # Adding messages with character limit check
        current_chars = len(self.summarized_history) if self.summarized_history else 0

        for msg in recent_messages:
            msg_formatted = {"role": msg["role"], "content": msg["content"]}
            msg_length = len(msg["content"])

            if max_chars and current_chars + msg_length > max_chars:
                # Truncate the last message if needed
                remaining_chars = max_chars - current_chars
                if remaining_chars > 50:  # Only add if meaningful content remains
                    msg_formatted["content"] = msg["content"][:remaining_chars] + "..."
                    messages.append(msg_formatted)
                break
            else:
                messages.append(msg_formatted)
                current_chars += msg_length

        return messages

    def summarize_conversation(self) -> str:
        """Summarize the current conversation history using Groq API."""
        if not self.conversation_history:
            return ""

        # Prepare conversation for summarization
        conversation_text = "\n".join([
            f"{msg['role'].upper()}: {msg['content']}"
            for msg in self.conversation_history
        ])

        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that creates concise summaries. Summarize the following conversation, capturing key points, decisions, and important information."
                    },
                    {
                        "role": "user",
                        "content": f"Please summarize this conversation:\n\n{conversation_text}"
                    }
                ],
                max_tokens=200,
                temperature=0.5
            )

            summary = response.choices[0].message.content
            return summary

        except Exception as e:
            print(f"Error during summarization: {str(e)}")
            return "Error creating summary"

    def check_and_perform_periodic_summarization(self):
        """Check if it's time for periodic summarization and perform if needed."""
        self.conversation_count += 1

        if self.conversation_count % self.summarize_every_k == 0:
            print(f"\n🔄 Performing periodic summarization (every {self.summarize_every_k} conversations)...")

            # Create summary
            new_summary = self.summarize_conversation()

            # Combine with existing summary if present
            if self.summarized_history:
                combined_context = f"Previous summary: {self.summarized_history}\nNew conversations: {new_summary}"

                # Get a combined summary
                try:
                    response = client.chat.completions.create(
                        model=self.model,
                        messages=[
                            {
                                "role": "system",
                                "content": "Combine these summaries into one concise summary."
                            },
                            {
                                "role": "user",
                                "content": combined_context
                            }
                        ],
                        max_tokens=250,
                        temperature=0.5
                    )
                    self.summarized_history = response.choices[0].message.content
                except:
                    self.summarized_history = new_summary
            else:
                self.summarized_history = new_summary

            # Store summary for demonstration
            self.all_summaries.append({
                "after_conversation": self.conversation_count,
                "summary": self.summarized_history,
                "timestamp": datetime.now().isoformat()
            })

            # Clearing old conversation history after summarization
            self.conversation_history = self.conversation_history[-2:]  # Keep last exchange

            print(f"✅ Summary created and stored!")
            print(f"📝 Summary: {self.summarized_history[:150]}...")

            return True
        return False

    def display_status(self):
        """Display current conversation status."""
        print("\n" + "="*50)
        print("📊 CONVERSATION STATUS")
        print("="*50)
        print(f"Total conversations: {self.conversation_count}")
        print(f"Messages in current history: {len(self.conversation_history)}")
        print(f"Summarization scheduled every: {self.summarize_every_k} conversations")
        print(f"Has summary: {'Yes' if self.summarized_history else 'No'}")
        print(f"Total summaries created: {len(self.all_summaries)}")
        print("="*50 + "\n")

print("✅ ConversationManager class created successfully!")


✅ ConversationManager class created successfully!


### 3.2 Demonstration Functions
Functions to demonstrate various truncation options and periodic summarization.

In [4]:
def demonstrate_truncation_options():
    """Demonstrate different truncation options with sample conversations."""

    print("🎯 DEMONSTRATING TRUNCATION OPTIONS\n")

    # Create a manager with sample conversations
    manager = ConversationManager(summarize_every_k=10)  # High k to avoid auto-summarization

    # Adding sample conversation
    sample_exchanges = [
        ("user", "Hi, I'm looking for a good Italian restaurant in downtown."),
        ("assistant", "I'd be happy to help you find an Italian restaurant downtown! What's your budget range?"),
        ("user", "Something mid-range, around $30-50 per person."),
        ("assistant", "Great! I recommend 'Bella Vista' on Main Street. They have excellent pasta and a cozy atmosphere."),
        ("user", "Do they have vegetarian options?"),
        ("assistant", "Yes, Bella Vista has several vegetarian options including eggplant parmigiana and various pasta dishes."),
        ("user", "Perfect! What are their hours?"),
        ("assistant", "They're open Tuesday-Sunday, 11:30 AM to 10 PM. Closed on Mondays."),
        ("user", "Can I make a reservation online?"),
        ("assistant", "Yes, you can make reservations through their website or by calling them directly.")
    ]

    # Adding messages to manager
    for role, content in sample_exchanges:
        manager.add_message(role, content)

    print("📝 Original conversation has", len(sample_exchanges), "messages\n")

    # Demonstrate truncation by turns
    print("1️⃣ TRUNCATION BY NUMBER OF TURNS:")
    print("-" * 40)

    for max_turns in [2, 4, 6]:
        truncated = manager.get_conversation_for_api(max_turns=max_turns)
        print(f"\n✂️  Last {max_turns} turns:")
        print(f"   Messages included: {len(truncated)}")
        print(f"   First message: {truncated[0]['content'][:60]}...")
        print(f"   Last message: {truncated[-1]['content'][:60]}...")

    # Demonstrate truncation by character length
    print("\n\n2️⃣ TRUNCATION BY CHARACTER LENGTH:")
    print("-" * 40)

    for max_chars in [200, 400, 600]:
        truncated = manager.get_conversation_for_api(max_chars=max_chars)
        total_chars = sum(len(msg['content']) for msg in truncated)
        print(f"\n✂️  Max {max_chars} characters:")
        print(f"   Messages included: {len(truncated)}")
        print(f"   Total characters: {total_chars}")
        print(f"   Last message: {truncated[-1]['content'][:60]}...")

    # Demonstrate combined truncation
    print("\n\n3️⃣ COMBINED TRUNCATION (turns + chars):")
    print("-" * 40)

    truncated = manager.get_conversation_for_api(max_turns=4, max_chars=300)
    total_chars = sum(len(msg['content']) for msg in truncated)
    print(f"\n✂️  Max 4 turns AND 300 characters:")
    print(f"   Messages included: {len(truncated)}")
    print(f"   Total characters: {total_chars}")

    return manager


def demonstrate_periodic_summarization():
    """Demonstrate periodic summarization after every k conversations."""

    print("\n\n🔄 DEMONSTRATING PERIODIC SUMMARIZATION\n")
    print("="*60)

    # Creating manager with summarization every 3 conversations
    manager = ConversationManager(summarize_every_k=3)

    # Sample conversation scenarios
    conversation_scenarios = [
        # Conversation 1: Restaurant inquiry
        [
            ("user", "I need a restaurant recommendation for a business dinner."),
            ("assistant", "I'd recommend The Executive Lounge for business dinners. It has private dining areas and excellent service."),
            ("user", "What's the dress code?"),
            ("assistant", "Business formal is required. Jackets for men, and business attire for all guests.")
        ],

        # Conversation 2: Travel planning
        [
            ("user", "I'm planning a trip to Paris next month. Any advice?"),
            ("assistant", "Paris is wonderful! I recommend visiting the Eiffel Tower early morning to avoid crowds."),
            ("user", "What about accommodation?"),
            ("assistant", "The Marais district is great for tourists - central location and many boutique hotels.")
        ],

        # Conversation 3: Tech support (triggers summarization)
        [
            ("user", "My laptop keeps freezing. What should I do?"),
            ("assistant", "First, try restarting in safe mode. Also check for any pending system updates."),
            ("user", "It's still slow after updates."),
            ("assistant", "You might need to check for malware or consider increasing your RAM.")
        ],

        # Conversation 4: Fitness advice
        [
            ("user", "I want to start working out. Any beginner tips?"),
            ("assistant", "Start with 3 days a week, focusing on full-body exercises. Don't forget to warm up!"),
            ("user", "What about diet?"),
            ("assistant", "Focus on protein intake and stay hydrated. Avoid drastic diet changes initially.")
        ],

        # Conversation 5: Book recommendations
        [
            ("user", "Can you recommend some sci-fi books?"),
            ("assistant", "Try 'Dune' by Frank Herbert or 'The Expanse' series by James S.A. Corey."),
            ("user", "Something more recent?"),
            ("assistant", "Andy Weir's 'Project Hail Mary' is excellent and was published in 2021.")
        ],

        # Conversation 6: Cooking help (triggers summarization)
        [
            ("user", "How do I make a perfect omelette?"),
            ("assistant", "Use medium-low heat, beat eggs thoroughly, and add fillings when eggs are slightly wet on top."),
            ("user", "My omelettes always break when I flip them."),
            ("assistant", "Try the fold method instead of flipping - just fold in half when bottom is set.")
        ]
    ]

    # Processing each conversation
    for i, conversation in enumerate(conversation_scenarios, 1):
        print(f"\n💬 CONVERSATION {i}")
        print("-" * 40)

        # Add messages from this conversation
        for role, content in conversation:
            manager.add_message(role, content)
            print(f"{role.upper()}: {content}")

        # Check for periodic summarization
        summarized = manager.check_and_perform_periodic_summarization()

        if not summarized:
            print(f"\n📌 No summarization yet. Waiting for conversation {manager.summarize_every_k}")

    # Display final status
    manager.display_status()

    # Show all summaries created
    print("\n📚 ALL SUMMARIES CREATED:")
    print("="*60)
    for i, summary_info in enumerate(manager.all_summaries, 1):
        print(f"\nSummary #{i} (After conversation {summary_info['after_conversation']}):")
        print(f"Created at: {summary_info['timestamp']}")
        print(f"Content: {summary_info['summary']}")
        print("-"*60)

    return manager


# Execute demonstrations
print("Ready to run demonstrations! Use the functions below.")


Ready to run demonstrations! Use the functions below.


### 3.3 Task 1 Demonstrations
Running demonstrations to show truncation options and periodic summarization functionality.


In [5]:
# Running Demonstration 1: Truncation Options
print("DEMONSTRATION 1: TRUNCATION OPTIONS")
print("=" * 80)
manager1 = demonstrate_truncation_options()


DEMONSTRATION 1: TRUNCATION OPTIONS
🎯 DEMONSTRATING TRUNCATION OPTIONS

📝 Original conversation has 10 messages

1️⃣ TRUNCATION BY NUMBER OF TURNS:
----------------------------------------

✂️  Last 2 turns:
   Messages included: 2
   First message: Can I make a reservation online?...
   Last message: Yes, you can make reservations through their website or by c...

✂️  Last 4 turns:
   Messages included: 4
   First message: Perfect! What are their hours?...
   Last message: Yes, you can make reservations through their website or by c...

✂️  Last 6 turns:
   Messages included: 6
   First message: Do they have vegetarian options?...
   Last message: Yes, you can make reservations through their website or by c...


2️⃣ TRUNCATION BY CHARACTER LENGTH:
----------------------------------------

✂️  Max 200 characters:
   Messages included: 3
   Total characters: 191
   Last message: Something mid-range, around $30-50 per person....

✂️  Max 400 characters:
   Messages included: 6
   Total c

In [6]:
# Running Demonstration 2: Periodic Summarization
print("\n\nDEMONSTRATION 2: PERIODIC SUMMARIZATION")
print("=" * 80)
manager2 = demonstrate_periodic_summarization()




DEMONSTRATION 2: PERIODIC SUMMARIZATION


🔄 DEMONSTRATING PERIODIC SUMMARIZATION


💬 CONVERSATION 1
----------------------------------------
USER: I need a restaurant recommendation for a business dinner.
ASSISTANT: I'd recommend The Executive Lounge for business dinners. It has private dining areas and excellent service.
USER: What's the dress code?
ASSISTANT: Business formal is required. Jackets for men, and business attire for all guests.

📌 No summarization yet. Waiting for conversation 3

💬 CONVERSATION 2
----------------------------------------
USER: I'm planning a trip to Paris next month. Any advice?
ASSISTANT: Paris is wonderful! I recommend visiting the Eiffel Tower early morning to avoid crowds.
USER: What about accommodation?
ASSISTANT: The Marais district is great for tourists - central location and many boutique hotels.

📌 No summarization yet. Waiting for conversation 3

💬 CONVERSATION 3
----------------------------------------
USER: My laptop keeps freezing. What shou

## 4. Task 2: JSON Schema Classification & Information Extraction

### 4.1 JSON Schema Definition and Function Calling Setup
Implementing structured data extraction using Groq's function calling capabilities.


In [7]:
# Defining the JSON schema for information extraction
information_schema = {
    "name": "extract_user_information",
    "description": "Extract user information from a conversation",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The user's full name"
            },
            "email": {
                "type": "string",
                "description": "The user's email address"
            },
            "phone": {
                "type": "string",
                "description": "The user's phone number"
            },
            "location": {
                "type": "object",
                "properties": {
                    "city": {
                        "type": "string",
                        "description": "The user's city"
                    },
                    "state": {
                        "type": "string",
                        "description": "The user's state or province"
                    },
                    "country": {
                        "type": "string",
                        "description": "The user's country"
                    }
                },
                "description": "The user's location details"
            },
            "age": {
                "type": "integer",
                "description": "The user's age"
            }
        },
        "required": []
    }
}

class InformationExtractor:
    """Extract structured information from conversations using Groq function calling."""

    def __init__(self, model: str = MODEL):
        self.model = model
        self.schema = information_schema

    def extract_information(self, conversation: str) -> Dict[str, Any]:
        """
        Extract information from a conversation using function calling.

        Args:
            conversation: The conversation text to analyze

        Returns:
            Dictionary containing extracted information
        """
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=[
                    {
                        "role": "system",
                        "content": "You are a helpful assistant that extracts user information from conversations. Extract only explicitly stated information."
                    },
                    {
                        "role": "user",
                        "content": f"Extract user information from this conversation:\n\n{conversation}"
                    }
                ],
                tools=[{
                    "type": "function",
                    "function": self.schema
                }],
                tool_choice={"type": "function", "function": {"name": "extract_user_information"}},
                temperature=0
            )

            # Extract the function call result
            if response.choices[0].message.tool_calls:
                tool_call = response.choices[0].message.tool_calls[0]
                extracted_data = json.loads(tool_call.function.arguments)
                return extracted_data
            else:
                return {}

        except Exception as e:
            print(f"Error during extraction: {str(e)}")
            return {}

    def validate_against_schema(self, data: Dict[str, Any]) -> Tuple[bool, List[str]]:
        """
        Validate extracted data against the schema.

        Args:
            data: The extracted data dictionary

        Returns:
            Tuple of (is_valid, list_of_errors)
        """
        errors = []

        # Check data types
        if 'name' in data and not isinstance(data['name'], str):
            errors.append("Name must be a string")

        if 'email' in data:
            if not isinstance(data['email'], str):
                errors.append("Email must be a string")
            elif '@' not in data['email']:
                errors.append("Email format appears invalid")

        if 'phone' in data and not isinstance(data['phone'], str):
            errors.append("Phone must be a string")

        if 'age' in data:
            if not isinstance(data['age'], int):
                errors.append("Age must be an integer")
            elif data['age'] < 0 or data['age'] > 150:
                errors.append("Age value seems unrealistic")

        if 'location' in data:
            if not isinstance(data['location'], dict):
                errors.append("Location must be an object")
            else:
                loc = data['location']
                for field in ['city', 'state', 'country']:
                    if field in loc and not isinstance(loc[field], str):
                        errors.append(f"Location {field} must be a string")

        is_valid = len(errors) == 0
        return is_valid, errors

    def display_extraction_result(self, conversation: str, extracted_data: Dict[str, Any],
                                validation_result: Tuple[bool, List[str]]):
        """Display the extraction results in a formatted way."""
        print("\n" + "="*60)
        print("📝 CONVERSATION:")
        print("-"*60)
        print(conversation)

        print("\n🔍 EXTRACTED INFORMATION:")
        print("-"*60)
        if extracted_data:
            print(json.dumps(extracted_data, indent=2))
        else:
            print("No information extracted")

        print("\n✅ VALIDATION RESULT:")
        print("-"*60)
        is_valid, errors = validation_result
        if is_valid:
            print("✓ Data is valid according to schema")
        else:
            print("✗ Validation errors found:")
            for error in errors:
                print(f"  - {error}")
        print("="*60)

print("✅ InformationExtractor class and schema created successfully!")


✅ InformationExtractor class and schema created successfully!


### 4.2 Sample Conversations for Information Extraction
Testing the extraction system with diverse conversation examples.


In [8]:
# Creating sample conversations for testing
sample_conversations = [
    # Conversation 1: Complete information
    """
    Customer: Hi, I'd like to create an account.
    Support: Sure! I'd be happy to help you create an account. May I have your full name?
    Customer: Yes, my name is John Michael Smith.
    Support: Thank you, Mr. Smith. And what email address would you like to use?
    Customer: Please use john.smith@email.com
    Support: Perfect. Can I also get a phone number for account security?
    Customer: Sure, it's +1-555-123-4567
    Support: Great! And where are you located?
    Customer: I'm in San Francisco, California, United States.
    Support: Excellent. One last thing - may I ask your age for our demographics?
    Customer: I'm 28 years old.
    Support: Thank you! Your account has been created successfully.
    """,

    # Conversation 2: Partial information
    """
    User: Hello, I'm interested in your services.
    Agent: Welcome! I'd be glad to help. What's your name?
    User: I'm Sarah Johnson.
    Agent: Nice to meet you, Sarah. How can I reach you?
    User: You can email me at sarah.j@gmail.com
    Agent: Thanks! Where are you based?
    User: I'm in Toronto, Canada.
    Agent: Great! Do you mind sharing your age group for our records?
    User: I'd prefer not to share that information.
    Agent: No problem at all! Let me show you our services.
    """,

    # Conversation 3: Information scattered throughout casual conversation
    """
    Customer: Hi there! I just moved to Austin, Texas from New York.
    Sales: Welcome to Austin! How are you finding it so far?
    Customer: It's great! Much warmer than NYC. By the way, I'm David Lee.
    Sales: Nice to meet you, David! What brings you to our store today?
    Customer: I need to update my contact information. I'm 35 now and figured it's time to get organized.
    Sales: Absolutely! What's the best way to reach you?
    Customer: My phone is 512-789-0123. That's the best way.
    Sales: Got it. Do you have an email as well?
    Customer: Oh yes, david.lee.35@hotmail.com - I know, showing my age with Hotmail!
    Sales: Haha, no judgment here! Let me update your information.
    Customer: Actually, I should mention I'm originally from South Korea, moved to the US five years ago.
    Sales: Thanks for sharing! Is Austin, Texas your permanent address now?
    Customer: Yes, Austin, Texas, USA is home now!
    """
]

# Creating the extractor
extractor = InformationExtractor()

# Processing each conversation
print("🎯 TESTING INFORMATION EXTRACTION ON SAMPLE CONVERSATIONS\n")

for i, conversation in enumerate(sample_conversations, 1):
    print(f"\n{'='*80}")
    print(f"📋 TEST CASE {i}")
    print('='*80)

    # Extracting information
    extracted_data = extractor.extract_information(conversation)

    # Validating the extracted data
    validation_result = extractor.validate_against_schema(extracted_data)

    # Displaying results
    extractor.display_extraction_result(conversation, extracted_data, validation_result)

    # Small delay to avoid rate limiting
    time.sleep(1)


🎯 TESTING INFORMATION EXTRACTION ON SAMPLE CONVERSATIONS


📋 TEST CASE 1

📝 CONVERSATION:
------------------------------------------------------------

    Customer: Hi, I'd like to create an account.
    Support: Sure! I'd be happy to help you create an account. May I have your full name?
    Customer: Yes, my name is John Michael Smith.
    Support: Thank you, Mr. Smith. And what email address would you like to use?
    Customer: Please use john.smith@email.com
    Support: Perfect. Can I also get a phone number for account security?
    Customer: Sure, it's +1-555-123-4567
    Support: Great! And where are you located?
    Customer: I'm in San Francisco, California, United States.
    Support: Excellent. One last thing - may I ask your age for our demographics?
    Customer: I'm 28 years old.
    Support: Thank you! Your account has been created successfully.
    

🔍 EXTRACTED INFORMATION:
------------------------------------------------------------
{
  "age": 28,
  "email": "john.s

## 5. Project Summary and Results

### Task 1: Conversation Management ✅
Successfully implemented:
- **Truncation by turns**: Limit conversation history by number of messages
- **Truncation by character length**: Control memory usage with character limits
- **Combined truncation**: Apply both turn and character limits simultaneously
- **Periodic summarization**: Automatically summarize every k conversations
- **Summary consolidation**: Merge new summaries with existing ones

### Task 2: Information Extraction ✅
Successfully implemented:
- **JSON Schema definition**: Structured schema for 5 information fields
- **Function calling**: Used Groq's OpenAI-compatible function calling
- **Robust extraction**: Handled complete, partial, and scattered information
- **Schema validation**: Validated all extracted data against defined schema

### Key Features
1. **No frameworks used** - Only standard Python libraries and OpenAI client
2. **Clean, documented code** - Every function and class is well-documented
3. **Comprehensive demonstrations** - Multiple test cases showing various scenarios
4. **Error handling** - Graceful error handling throughout the implementation

### Technical Implementation
- **API**: Groq API with OpenAI SDK compatibility
- **Model**: llama-3.1-8b-instant
- **Languages**: Python 3.x
- **Environment**: Google Colab

---

This project demonstrates proficiency in:
- API integration and management
- Conversation history handling
- Natural language processing
- Structured data extraction
- Clean code practices


In [9]:
# Final confirmation
print("🎉 PROJECT COMPLETED SUCCESSFULLY!")
print("\n📊 Final Statistics:")
print(f"- Task 1: Conversation Management with {len(manager2.all_summaries)} successful summarizations")
print(f"- Task 2: Information Extraction with {len(sample_conversations)} test cases validated")
print("\n✅ All requirements met:")
print("- No frameworks used")
print("- Groq API with OpenAI SDK")
print("- Clean, documented code")
print("- Comprehensive demonstrations")
print("- API key included for testing")

🎉 PROJECT COMPLETED SUCCESSFULLY!

📊 Final Statistics:
- Task 1: Conversation Management with 2 successful summarizations
- Task 2: Information Extraction with 3 test cases validated

✅ All requirements met:
- No frameworks used
- Groq API with OpenAI SDK
- Clean, documented code
- Comprehensive demonstrations
- API key included for testing
